# PRUEBA DE CARGO PARA LIDER DE LINEA DE CONOCIMIENTO EN IA

## Limpieza de datos y validación de consistencia

In [1]:
# import modules needed for the analysis
import pandas as pd
import numpy as np
import os

In [2]:
#############################################
###### Parameters for the execution #########
#############################################

## possible phantom nulls list
PHANTOM_NULLS = ['-','','<na>','nA','nan','null','_','vacio', 'NaN']

# Get the absolute path to the directory where the script is located
script_dir = os.getcwd()
script_dir = script_dir.replace('\\','/')

# construct the paths needed for the execution of the code:
data__raw_path = f"{script_dir}/../../data/raw/restaurants_dataset.csv"
data_staging_path = f"{script_dir}/../../data/staging/restaurants_dataset.parquet"

In [3]:
# load raw data 
raw_data = pd.read_csv(data__raw_path,sep=',')

# Describe the shape and complexity of the dataframe
print("dimensiones de la base de datos: ", raw_data.shape)
print("cuantos valores únicos tienen las 10 variables con mayor variabilidad \n",raw_data.nunique().sort_values(ascending=False).head(10))

dimensiones de la base de datos:  (3493, 34)
cuantos valores únicos tienen las 10 variables con mayor variabilidad 
 Registration Number              3493
Opening Day of Restaurant        1778
Facebook Popularity Quotient      776
Instagram Popularity Quotient     742
City                              297
Annual Turnover                   173
Restaurant Theme                   34
Cuisine                            20
Service                            11
Lively                             11
dtype: int64


Se observa que la variable "Registration Number" es la llave primaria de la tabla, ademas, las columnas que cuentan con mas variabilidad en la tabla son:
- Registration Number
- Opening Day of Restaurant
- Facebook Popularity Quotient
- Instagram Popularity Quotient
- City

Lo cual es muy importante al momento de hacer el diseño de variables para los modelos que se van a generar en la parte 2 del desafio

In [4]:
## Get and analize the number of NA
print("Número de campos vacios en el top 10 de columnas con mas campos vacios")
raw_data.isnull().sum().sort_values(ascending=False).head(10)

Número de campos vacios en el top 10 de columnas con mas campos vacios


Live Sports Rating               3288
Value Deals Rating               2707
Comedy Gigs Rating               2483
Live Music Rating                 765
Overall Restaurant Rating         212
Facebook Popularity Quotient       99
Instagram Popularity Quotient      56
Resturant Tier                     49
Ambience                           25
Registration Number                 0
dtype: int64

In [5]:
## function to analize the consistency and state of the table's data
def analize_cols(df):
    df=df.copy()
    
    columnas = ['nombre','cantidad_de_valores_unicos','cantidad_nulos','cantidad_nulos_fantasma','tipo_de_dato','convertible_float','convertible_int','cantidad_ceros','cantidad_negativos','cantidad_positivos']
    df_analisis_columnas = pd.DataFrame(columns=columnas)
    contador=1
    #print(len(df.columns))
    for col in df.columns:      
        
        # print(str(contador)+'/'+str(cantidad_de_columnas)+'   '+ col)
        contador=contador+1
        nombre = col       
        cantidad_de_valores_unicos = len(df[col].unique())        
        cantidad_nulos = df[col].isna().sum()
        
        # transform the column to lower case and search for phantom nulls
        contador_vacios_fantasma= pd.Series([str(i).lower for i in df[col]]).isin(PHANTOM_NULLS).sum()

        convertible_int='False'
        try:
            float_val = df[col].astype(float)
            convertible_float = 'True'
            
            cantidad_ceros = sum(float_val.fillna(1)==0.0)
            cantidad_negativos = sum(float_val.fillna(0)<0.0)
            cantidad_positivos  = sum(float_val.fillna(0)>0.0)
            
        except:
            convertible_float = 'False'            
            cantidad_ceros = pd.NA
            cantidad_negativos = pd.NA
            cantidad_positivos  = pd.NA 

        try:
            int_val = df[col].astype('Int64')
            convertible_int = 'True'  
        except:
            convertible_int = 'False'
            
        # add values to the dataframe       
        new_row = {
                    'nombre': nombre,
                    'cantidad_de_valores_unicos': cantidad_de_valores_unicos,
                    'cantidad_nulos': cantidad_nulos,
                    'cantidad_nulos_fantasma': contador_vacios_fantasma,
                    'tipo_de_dato': df[col].dtype,
                    'convertible_float': convertible_float,
                    'convertible_int': convertible_int,
                    'cantidad_ceros': cantidad_ceros,
                    'cantidad_negativos': cantidad_negativos,
                    'cantidad_positivos': cantidad_positivos
                  }

        # Use the loc method to add the new row to the DataFrame
        df_analisis_columnas.loc[len(df_analisis_columnas)] = new_row    
        
    return df_analisis_columnas

In [6]:
# Report for the consistency of the table
df_analized_columns = analize_cols(raw_data)
print("Reporte con el estado de cada una de las variables de la tabla y analisis de consistencia")
df_analized_columns.sort_values("cantidad_de_valores_unicos", ascending=False)

Reporte con el estado de cada una de las variables de la tabla y analisis de consistencia


,nombre,cantidad_de_valores_unicos,cantidad_nulos,cantidad_nulos_fantasma,tipo_de_dato,convertible_float,convertible_int,cantidad_ceros,cantidad_negativos,cantidad_positivos
0,Registration Number,3493,0,0,int64,True,True,0,0,3493
5,Opening Day of Restaurant,1778,0,0,object,False,False,NaN,NaN,NaN
6,Facebook Popularity Quotient,777,99,0,float64,True,False,0,0,3394
8,Instagram Popularity Quotient,743,56,0,float64,True,False,0,0,3437
3,City,297,0,0,object,False,False,NaN,NaN,NaN
1,Annual Turnover,173,0,0,int64,True,True,0,0,3493
16,Restaurant Theme,34,0,0,object,False,False,NaN,NaN,NaN
2,Cuisine,20,0,0,object,False,False,<NA>,<NA>,<NA>
29,Ambience,12,25,0,float64,True,True,21,0,3447
30,Lively,11,0,0,int64,True,True,29,0,3464


Se observa que en general la tabla es bastante consistente, ya que:
-  no presenta ningun nulo fantasma
-  no hay variables que esten de tipo object o string y en realidad sean numericas
-  Todas las variables numericas estan definidas en el rango de los positivos

sin embargo, se debe revisar mas a fondo los valores de las variables que son de tipo float pero tienen pocos valores unicos como es el caso de "Resturant Tier", ya que en caso de que sean numeros enteros, es mejor transformarlas a int antes de llevar los datos a la zona de staging.

Ademas, se observa una cantidad importante de valores nulos particularmente en 9 variables de la tabla, por lo que se tiene que tomar la decision de si se imputaran, se eliminaran o se dejan de esa manera los registros.

In [7]:
# variables with data type float and less than 20 unique values
float_int_variables = df_analized_columns[(df_analized_columns["tipo_de_dato"] == "float64") & (df_analized_columns["cantidad_de_valores_unicos"] <= 20)]["nombre"]
# unique values for the previous variables
print("top 7 de valores con mas concurrencia en las variables float con pocos valores unicos")
for col_name in float_int_variables:
    print(raw_data[col_name].value_counts().head(5))

top 7 de valores con mas concurrencia en las variables float con pocos valores unicos
Resturant Tier
2.0    3191
1.0     253
Name: count, dtype: int64
Overall Restaurant Rating
9.0     1016
10.0     859
7.0      585
8.0      533
6.0      288
Name: count, dtype: int64
Live Music Rating
4.0    1110
3.0     684
5.0     629
6.0     162
2.0      97
Name: count, dtype: int64
Comedy Gigs Rating
3.0    472
2.0    278
4.0    191
5.0     38
1.0     30
Name: count, dtype: int64
Value Deals Rating
3.0    280
4.0    279
5.0    127
2.0     75
6.0     19
Name: count, dtype: int64
Live Sports Rating
3.0    79
4.0    75
5.0    27
2.0    20
6.0     4
Name: count, dtype: int64
Ambience
6.0    679
8.0    665
7.0    596
5.0    368
4.0    349
Name: count, dtype: int64


Vemos que todas las variables mostradas anteriormente son en realidad variables de tipo entero, por lo que se van a convertir antes de llevarlas a la zona de staging

In [8]:
# Analisys of null count
print("Cantidad de nulos en las variables que contienen dichos registros:")
null_count_table = raw_data.isnull().sum().sort_values(ascending=False).head(9).reset_index()
null_count_table.columns = ["variable", "cantidad de nulos"]
null_count_table

Cantidad de nulos en las variables que contienen dichos registros:


,variable,cantidad de nulos
0,Live Sports Rating,3288
1,Value Deals Rating,2707
2,Comedy Gigs Rating,2483
3,Live Music Rating,765
4,Overall Restaurant Rating,212
5,Facebook Popularity Quotient,99
6,Instagram Popularity Quotient,56
7,Resturant Tier,49
8,Ambience,25


In [9]:
# frequency analisys for variables with nulls
print("top 7 de valores con mas concurrencia en las variables con valores nulos ")
for col_name in null_count_table.variable:
    print(raw_data[col_name].value_counts().head(7))
    print("numero de nulos:",sum(raw_data[col_name].isnull()))

top 7 de valores con mas concurrencia en las variables con valores nulos 
Live Sports Rating
3.0    79
4.0    75
5.0    27
2.0    20
6.0     4
Name: count, dtype: int64
numero de nulos: 3288
Value Deals Rating
3.0    280
4.0    279
5.0    127
2.0     75
6.0     19
1.0      4
7.0      2
Name: count, dtype: int64
numero de nulos: 2707
Comedy Gigs Rating
3.0    472
2.0    278
4.0    191
5.0     38
1.0     30
6.0      1
Name: count, dtype: int64
numero de nulos: 2483
Live Music Rating
4.0    1110
3.0     684
5.0     629
6.0     162
2.0      97
1.0      30
7.0      14
Name: count, dtype: int64
numero de nulos: 765
Overall Restaurant Rating
9.0     1016
10.0     859
7.0      585
8.0      533
6.0      288
Name: count, dtype: int64
numero de nulos: 212
Facebook Popularity Quotient
82.0    62
75.0    61
76.0    59
78.0    59
72.0    58
73.0    58
87.0    56
Name: count, dtype: int64
numero de nulos: 99
Instagram Popularity Quotient
70.0    64
72.0    60
74.0    56
65.0    52
68.0    51
60.0    

Dado el analisis anterior, es posible que en el procesamiento que se hace desde la zona de staging hacia la zona de analitycs se decida eliminar la variable. Sin embargo, dado que este proceso se enfoca en limpiar la tabla, en este momento la variable se va a mantener.

## Cleanup process

### functions for cleaning the table

In [10]:
from CleanupFunc import CleanUp

# init the cleanup function
CleanData = CleanUp(raw_data)
CleanData.remove_lead_spaces()
CleanData.convert_lower_case()
# convert float variables to int
CleanData.convert_float_to_int(cols=list(float_int_variables))
# convert the date cols to datetime
CleanData.convert_date_cols(cols=["Opening Day of Restaurant"])

In [11]:
raw_data.describe()

,Registration Number,Annual Turnover,Opening Day of Restaurant,Facebook Popularity Quotient,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,Dedicated Parking,Open Sitting Available,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
count,3493.000000,3.493000e+03,3493,3394.000000,3437.000000,3493.000000,3493.000000,3493.000000,3493.000000,3493.000000,...,3281.0,2728.0,1010.0,786.0,205.0,3468.0,3493.000000,3493.000000,3493.00000,3493.000000
mean,61747.000000,3.072571e+07,2009-11-29 16:39:42.822788352,77.938715,74.404684,0.788720,0.988262,0.808188,0.801889,0.800172,...,8.479427,4.01283,2.932673,3.655216,3.590244,6.42301,6.874893,4.546808,3.23132,6.275122
min,60001.000000,3.500000e+06,1996-10-25 00:00:00,43.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.0,1.0,1.0,1.0,2.0,0.0,0.000000,0.000000,0.00000,0.000000
25%,60874.000000,1.800000e+07,2008-11-08 00:00:00,72.000000,66.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,7.0,3.0,2.0,3.0,3.0,5.0,6.000000,3.000000,2.00000,5.000000
50%,61747.000000,3.000000e+07,2010-02-28 00:00:00,79.000000,74.050000,1.000000,1.000000,1.000000,1.000000,1.000000,...,9.0,4.0,3.0,4.0,4.0,7.0,7.000000,5.000000,3.00000,6.000000
75%,62620.000000,3.700000e+07,2011-03-06 00:00:00,85.745000,82.400000,1.000000,1.000000,1.000000,1.000000,1.000000,...,10.0,5.0,3.0,4.0,4.0,8.0,8.000000,6.000000,5.00000,8.000000
max,63493.000000,4.000000e+08,2016-05-22 00:00:00,97.760000,98.700000,1.000000,1.000000,1.000000,1.000000,1.000000,...,10.0,8.0,6.0,7.0,6.0,10.0,10.000000,10.000000,10.00000,10.000000
std,1008.486572,2.165125e+07,NaN,9.829169,10.940327,0.408275,0.107719,0.393783,0.398633,0.399928,...,1.287233,1.009044,0.859519,0.97325,0.906351,2.050026,1.847131,1.877063,1.99305,1.895057


Una vez realizadas las limpiezas correspondientes a la tabla que se va a usar como input para el EDA y los modelos supervisados, se procede a cargar la data modificada a la capa de staging, con el fin de iniciar el EDA correspondiente para para los datos.

In [12]:
# load the data in parquet format without partitiones given is doesn't have enough rows and doesn't have a proper column to be partitioned by.
raw_data.to_parquet(data_staging_path)